In this section all required libraries are imported

In [ ]:
import numpy as np
from geneticalgorithm import geneticalgorithm as ga
import csv
import pandas as pd
from keras.utils import np_utils
from sklearn import metrics
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import SGD,Adam
from keras.models import Model
from keras import layers
import keras
import sklearn.model_selection
from keras.initializers import RandomUniform
from keras import backend as back

# Preprocessing

Preprocessing including 4 steps:
1_ Normalization: Because the minimum and maximum of EMG signal is known, so the best method for normalization is minmax method. After normalization all data placed between -1 and +1.
2_ Windowing: For increasing the number of data and preparing them in order to feed them to model, windowing is crucial. Also, the length and shape of window and overlapping between consecutive windows are essential factors that cause variation in the accuracy of model. The length of window is an important factor that determines usecase of network for online and offline recognition.
3_ Splitting data into train, validation and test set with the rate of 70, 15 and 15 percent. Also, test data has not be seen by model before.
4_ Preparing target: The last layer of model in comprised of softmax layer. Therefor we need to transform labels into onehot form.

In [ ]:
#Data preprocessing with 16 channels and N samples
def mode_rows(a):
    a = np.ascontiguousarray(a)
    void_dt = np.dtype((np.void, a.dtype.itemsize * np.prod(a.shape[1:])))
    _,ids, count = np.unique(a.view(void_dt).ravel(), \
                                return_index=1,return_counts=1)
    largest_count_id = ids[count.argmax()]
    most_frequent_row = a[largest_count_id]
    return most_frequent_row

#import data from dataset file
a = pd.read_pickle(r'/content/dualmyo_dataset.pkl')
signal_list =a[0]
label_list =a[1]

signal = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
k = 0
coef = np.ones([400,1])
target = np.array([0])

#windowing the signal
for s in signal_list :
  signal1 = np.hstack([abs(s[:,2:10]),abs(s[:,12:20])])
  signal = np.vstack((signal,signal1))
  target = np.vstack((target,label_list[k]*coef))
  k = k + 1

print(signal.shape)
print(target.shape)

data = np.concatenate((signal,target),axis=1)

seq_len = 200
stride = 200
dataset=[]

for i in range(0 , len(data)-seq_len , stride):
    sample =  data[i:i+seq_len,:]
    dataset.append(sample)

#Splitting data into train and test sets
#np.random.shuffle(dataset)
train, test = sklearn.model_selection.train_test_split(dataset , train_size =int( np.round(0.85*len(dataset))) , test_size = int(np.round(0.15*len(dataset))), shuffle = False)


x_train = np.zeros([len(train),seq_len,16,1])
y_train = np.zeros([len(train),1])
f=0
for s  in train :
    x_train[f,:,:,:] = s[:,0:16].reshape(-1,16,1)
    y_train[f, 0] = mode_rows(s[seq_len-20:seq_len,16])
    f = f+1
#Normalizing input data and transforming output to onehot for train data
x_tra = (x_train/128).astype('float32')
y_tra = np_utils.to_categorical(y_train)



x_test = np.zeros([len(test),seq_len,16,1])
y_test = np.zeros([len(test),1])
p=0
for s  in test :
    x_test[p,:,:,:] = s[:,0:16].reshape(-1,16,1)
    y_test[p, 0] = mode_rows(s[seq_len-20:seq_len,16])
    p = p+1
#Normalizing input data and transforming output to onehot for test data
x_tes = (x_test/128).astype('float32')
y_tes = np_utils.to_categorical(y_test)

# Define function which create model with various hyper-parameters:
The output of this function is considered as cost function of genetic algorithm. Genetic algorithm tries to minimize the cost function. So, we use the minus of accuracy of model as cost function.

In [ ]:

noi=0
pp={}
def f(XX):
    global noi
    noi = noi+1
    v = tuple(map(tuple,[XX]))
    if v in pp:
        return(pp[v])
    else:
        a=int(XX[0])
        print((XX[0]))

        b=int(XX[1])
        print((XX[1]))

        c=int(XX[2])
        print((XX[2]))

        d=int(XX[3])
        print((XX[3]))

        e=int(XX[4])
        print((XX[4]))

        f=int(XX[5])
        print((XX[5]))

        c1 = int(XX[6])
        print(XX[6])
        c2 = int(XX[7])
        print(XX[7])

        d1 = int(XX[8])
        print(XX[8])
        d2 = int(XX[9])
        print(XX[9])

        e1 = int(XX[10])
        print(XX[10])
        e2 = int(XX[11])
        print(XX[11])

        f1 = int(XX[12])
        print(XX[12])
        f2 = int(XX[13])
        print(XX[13])




        if a == 1:

            myInput = layers.Input(shape=(seq_len,16,1))
            conv1 = layers.Conv2D(c, (c1,c2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(myInput)
            flat = layers.Flatten()(conv1)
            dense1 = layers.Dense(b, activation='relu',kernel_initializer=RandomUniform())(flat)
            out_layer = layers.Dense(8, activation='softmax',kernel_initializer=RandomUniform())(dense1)

        elif a==2:

            myInput = layers.Input(shape=(seq_len,16,1))
            conv1 = layers.Conv2D(c, (c1,c2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(myInput)
            conv2 = layers.Conv2D(d, (d1,d2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(conv1)
            flat = layers.Flatten()(conv2)
            dense1 = layers.Dense(b, activation='relu',kernel_initializer=RandomUniform())(flat)
            out_layer = layers.Dense(8, activation='softmax',kernel_initializer=RandomUniform())(dense1)

        elif a==3:

            myInput = layers.Input(shape=(seq_len,16,1))
            conv1 = layers.Conv2D(c, (c1,c2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(myInput)
            conv2 = layers.Conv2D(d, (d1,d2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(conv1)
            conv3 = layers.Conv2D(e, (e1,e2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(conv2)
            flat = layers.Flatten()(conv3)
            dense1 = layers.Dense(b, activation='relu',kernel_initializer=RandomUniform())(flat)
            out_layer = layers.Dense(8, activation='softmax',kernel_initializer=RandomUniform())(dense1)

        elif a==4:

            myInput = layers.Input(shape=(seq_len,16,1))
            conv1 = layers.Conv2D(c, (c1,c2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(myInput)
            conv2 = layers.Conv2D(d, (d1,d2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(conv1)
            conv3 = layers.Conv2D(e, (e1,e2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(conv2)
            conv4 = layers.Conv2D(f, (f1,f2), activation='relu',kernel_initializer=RandomUniform(), padding='same', strides=1)(conv3)
            flat = layers.Flatten()(conv4)
            dense1 = layers.Dense(b, activation='relu',kernel_initializer=RandomUniform())(flat)
            out_layer = layers.Dense(8, activation='softmax',kernel_initializer=RandomUniform())(dense1)





        model = Model(myInput, out_layer)

        #Compilie
        model.compile( optimizer=Adam (lr=0.001 ) , loss='categorical_crossentropy',metrics=['accuracy'])


        #model fit
        model.fit(x_tra , y_tra, epochs=15 , batch_size=64 ,shuffle=True , validation_split=0.15)

        #model evaluation
        test_loss, test_acc = model.evaluate(x_tes, y_tes)
        back.clear_session()
        pp[v] = -test_acc

        return(-test_acc)

In [ ]:
# Define genetic algorithm


In [ ]:
varbound = np.array([[1, 4 ],[1, 150],[1, 150],[1, 150],[1, 150],[1, 150],[1,64],[1,16],[1,64],[1,16],[1,64],[1,16],[1,64],[1,16]])
vartype = np.array([['int'],['int'],['int'],['int'],['int'],['int'],['int'],['int'],['int'],['int'],['int'],['int'],['int'],['int']])

algorithm_param = {'max_num_iteration':5000 ,\
                   'population_size':42,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.1,\
                   'crossover_probability': 0.2,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':5}
gmodel = ga(function=f,dimension=14,variable_type_mixed=vartype,variable_boundaries=varbound,
                              algorithm_parameters=algorithm_param,function_timeout = 720000)

gmodel.run()
convergence=gmodel.report
output = gmodel.output_dict
print('#####################################################')
print(-(output['function']))
print((output['variable']))